# Do logarithmic proximity measures outperform plain ones in graph clustering?

In [1]:
from collections import defaultdict
from itertools import combinations
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import adjusted_rand_score

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from py_graphs.graphs.generator import StochasticBlockModel
from py_graphs.measure import *
from py_graphs.cluster.ward import Ward

## 2. Logarithmic vs. plain measures
Let $G(N,(m)p_{in}, p_{out})$ be the model of generating random graphs on $N$ nodes divided into $m$ classes of the same size, with $p_{in}$ and pout being the probability of $(i, j) \in E(G)$ for $i$ and $j$ belonging to the same class and different classes, respectively, where $E(G)$ is the edge set of $G$.

The curves in Figures 1–3 present the adjusted Rand index (averaged over 200 random graphs) for clustering with Ward’s
method.

In [4]:
def classic_plot(measure_class, graphs, n_class):
    raw_param_dict = defaultdict(list)
    for graph_idx, (edges, nodes) in enumerate(tqdm(graphs, desc=measure_class.name)):
        measure = measure_class(edges)
        for param_flat in np.linspace(0, 1, 31):
            try:
                param = measure.scaler.scale(param_flat)
                K = measure.get_K(param)
                y_pred = Ward(n_class).predict(K)
                ari = adjusted_rand_score(nodes, y_pred)
                raw_param_dict[param_flat].append(ari)
            except Exception as e:
#                 print("{}, {:.2f}, graph {}: {}".format(measure_class.name, param, graph_idx, e))
                pass
    for param, values in raw_param_dict.items():
        raw_param_dict[param] = np.nanmean(values)
    return list(zip(*sorted(raw_param_dict.items(), key=lambda x: x[0])))

In [5]:
def draw(results):
    fig, ax = plt.subplots(1, 4, figsize=(15, 4))
    for idx, names in enumerate([
        [('pWalk H', 'pWalk'), ('Walk H', 'Walk')],
        [('For H', 'For'), ('logFor H', 'logFor')],
        [('Comm H', 'Comm'), ('logComm H', 'logComm')],
        [('Heat H', 'Heat'), ('logHeat H', 'logHeat')]
    ]):
        ax[idx].plot(results[names[0][0]][0], results[names[0][0]][1], label=names[0][1])
        ax[idx].plot(results[names[1][0]][0], results[names[1][0]][1], label=names[1][1])
        ax[idx].set_xlim(0, 1)
        ax[idx].set_ylim(0.2, 1)
        ax[idx].legend()

### **Fig. 1** Logarithmic vs. plain measures for G(100,(2)0.2,0.05)

In [ ]:
results = defaultdict(list)
graphs, info = StochasticBlockModel(100, 2, 0.2, 0.05).generate_graphs(200)
for measure_class in [pWalk_H, Walk_H, For_H, logFor_H, Comm_H, logComm_H, Heat_H, logHeat_H]:
    results[measure_class.name] = classic_plot(measure_class, graphs, 2)
with open('results/2_1.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
draw(results)

### **Fig. 2** Logarithmic vs. plain measures for G(100,(3)0.3,0.1)

In [ ]:
results = defaultdict(list)
graphs, info = StochasticBlockModel(100, 3, 0.3, 0.1).generate_graphs(200)
for measure_class in [pWalk_H, Walk_H, For_H, logFor_H, Comm_H, logComm_H, Heat_H, logHeat_H]:
    results[measure_class.name] = classic_plot(measure_class, graphs, 3)
with open('results/2_2.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
draw(results)

### **Fig. 3** Logarithmic vs. plain measures for G(200,(2)0.3,0.1)

In [ ]:
results = defaultdict(list)
graphs, info = StochasticBlockModel(200, 2, 0.3, 0.1).generate_graphs(100)
for measure_class in [pWalk_H, Walk_H, For_H, logFor_H, Comm_H, logComm_H, Heat_H, logHeat_H]:
    results[measure_class.name] = classic_plot(measure_class, graphs, 2)
with open('results/2_3.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
draw(results)

## 3. Competition by Copeland’s score
The competition has been performed on random graphs generated with the $G(N,(m)p_{in}, p_{out})$ model and the following parameters: $N \in {100, 200}$, the number of classes $m \in {2, 4}$, $p_{in} = 0.3$, $p_{out} \in {0.1, 0.15}$. For every combination of parameters, we generated 50 graphs and for each of them we computed the best ARI’s the measure families reached. The results are presented in Table 1(a).

### find best params and 95% percentile

In [ ]:
results = defaultdict(lambda: defaultdict(0))
for n_nodes in [100, 200]:
    for n_classes in [2, 4]:
        for p_out in [0.1, 0.15]:
            column = (n_nodes, n_classes, p_out)
            graphs, info = StochasticBlockModel(n_nodes, n_classes, 0.3, p_out).generate_graphs(50)
            for measure_class in H_kernels_plus_RSP_FE:
                results[column][measure_class.name] = classic_plot(measure_class, graphs, n_classes)

### calc competition for given params

In [ ]:
def copeland(results):
    scores = defaultdict(lambda: 0)
    for a, b in list(combinations(enumerate(results), 2)):
        if a[1] > b[1]:
            scores[a[0]] += 1 
            scores[b[0]] -= 1
        elif a[1] < b[1]:
            scores[a[0]] -= 1
            scores[b[0]] += 1
    return scores